In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import folium
import time
import seaborn as sns
import plotly.express as px




In [2]:
load_dotenv()

Open_Weather_API_Key_Myriam= os.getenv("Open_Weather_API_Key_Myriam") 

# 35 Best cities

Sur la liste des 35 plus belles villes, nous avons 34 noms de villes + 1 lieu d'interêt qui n'est pas une ville (Gorges du Verdon). Pour récupérer les latitudes et longitudes des 34 villes nous pouvons facilement utiliser l'API openweathermap. Cette API permet facilement de collecter également le pays et la région, ce qui est utile contrôler les éventuelles erreures liées à des villes homonymes.
Cependant, cette API ne reconnait pas les Gorges du Verdon. Pour palier ce problème, une seconde API a été utilisée, nominatim.openstreetmap, mais elle ne permet pas de récupérer la région et le pays.

In [10]:
best_cities= ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle",
#"Gorges du Verdon" exclu de la liste car n'est pas une ville mais un point d'intérêt.
]


In [11]:
len(best_cities)
type(best_cities)
best_cities[-1]

'La Rochelle'

In [12]:
df = pd.DataFrame(data = best_cities, columns = ["cities"])
df

,cities
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen
5,Paris
6,Amiens
7,Lille
8,Strasbourg
9,Chateau du Haut Koenigsbourg


## Import longitudes & latitudes of 35 best cities

In [13]:
df.shape

(34, 1)

In [14]:
for i,best_city in enumerate(best_cities):
    r = requests.get(f"http://api.openweathermap.org/geo/1.0/direct?q={best_city}&limit=1&appid={Open_Weather_API_Key_Myriam}")
    data = r.json()
  
    df.loc[i,"name"] = data[0]["name"] #pour s'assurer de la bonne attribution de la ville
    df.loc[i,"country"] = data[0]["country"] 
    df.loc[i,"state"] = data[0]["state"]
    df.loc[i,"lat"] = data[0]["lat"]
    df.loc[i,"lon"] = data[0]["lon"]

df

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.858890,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [16]:
import time

def rechercher_coordonnees(lieux):
    url = "https://nominatim.openstreetmap.org/search"
    coordonnees = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'    }

    for lieu in lieux:
        params = {
            'q': lieu,
            'format': 'json',
            'limit': 1
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()
            if data:
                address = data[0].get('address', {})
                coordonnees.append({
                    'cities': lieu,
                    'name': address.get('city'),
                    'country': address.get('country'),
                    'state': address.get('state'),
                    'lat': data[0].get('lat'),
                    'lon': data[0].get('lon'),
                })
            else:
                coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None

                })
        else:
            print(f"Erreur lors de la requête pour {lieu} :", response.status_code)
            coordonnees.append({
                    'cities': lieu,
                    'name': None,
                    'country': None,
                    'state': None,
                    'lat': None,
                    'lon': None,
            })

        # Attendre un peu entre les requêtes pour éviter de dépasser les limites
        time.sleep(1)

    return coordonnees


coordonnees = rechercher_coordonnees(["Gorges du Verdon"])
print(coordonnees)


[{'cities': 'Gorges du Verdon', 'name': None, 'country': None, 'state': None, 'lat': '43.7496562', 'lon': '6.3285616'}]


In [17]:
gdv = pd.DataFrame(coordonnees)
gdv

,cities,name,country,state,lat,lon
0,Gorges du Verdon,None,None,None,43.7496562,6.3285616


In [18]:
df_merged = pd.concat([df, gdv], ignore_index=True)
df_merged

,cities,name,country,state,lat,lon
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.51146
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966
5,Paris,Paris,FR,Ile-de-France,48.85889,2.320041
6,Amiens,Amiens,FR,Hauts-de-France,49.894171,2.295695
7,Lille,Lille,FR,Hauts-de-France,50.636565,3.063528
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492


In [19]:
# Enregistrer le DataFrame en fichier CSV
df_merged.to_csv('best_cities_merged.csv', index=False)

In [75]:
# Créer une carte centrée sur la France
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les villes à la carte
for i, row in df_merged.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities']
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('map_best_city.html')

# Afficher la carte
map

## Import weather information

In [21]:
df=df_merged

In [22]:
url = f"https://api.openweathermap.org/data/3.0/onecall?lat={df.loc[0, 'lat']}&lon={df.loc[0, 'lon']}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

r = requests.get(url)
data = r.json()
r

<Response [200]>

In [29]:
data["current"]

{'dt': 1743155553,
 'sunrise': 1743141120,
 'sunset': 1743186596,
 'temp': 9.73,
 'feels_like': 7.64,
 'pressure': 1016,
 'humidity': 93,
 'dew_point': 8.65,
 'uvi': 0.54,
 'clouds': 100,
 'visibility': 10000,
 'wind_speed': 4.07,
 'wind_deg': 251,
 'wind_gust': 8.71,
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}]}

In [31]:

for i in df.index:
    lat = df.loc[i, 'lat']
    lon = df.loc[i, 'lon']

    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&exclude=hourly,daily&appid={Open_Weather_API_Key_Myriam}"

    r = requests.get(url)

    if r.status_code == 200:
        data = r.json()
  
        df.loc[i,"date"] = data["current"]["dt"]
        df.loc[i, "temp"] = data["current"]["temp"]
        df.loc[i, "feels_like"] = data["current"]["feels_like"]
        df.loc[i, "uvi"] = data["current"]["uvi"]
        df.loc[i, "humidity"] = data["current"]["humidity"]
        df.loc[i, "wind_speed"] = data["current"]["wind_speed"]
        df.loc[i, "description"] = data["current"]["weather"][0]["description"]
        
    else:
        print(f"Erreur lors de la requête pour l'index {i} : {r.status_code}")
        print(r.text)  # Imprime le message d'erreur pour plus de détails



In [32]:
df.columns

Index(['cities', 'name', 'country', 'state', 'lat', 'lon', 'date', 'temp',
       'feels_like', 'uvi', 'humidity', 'wind_speed', 'description'],
      dtype='object')

In [33]:
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.51146,2025-03-28 09:56:08,9.73,7.64,0.54,93.0,4.07,overcast clouds
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041,2025-03-28 09:56:26,10.26,9.69,0.40,90.0,3.60,overcast clouds
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474,2025-03-28 09:56:26,9.25,6.52,0.64,93.0,5.34,overcast clouds
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973,2025-03-28 09:56:26,10.03,9.41,1.57,89.0,4.12,mist
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966,2025-03-28 09:56:26,10.43,9.67,3.20,82.0,1.79,overcast clouds


In [34]:
# Enregistrer le DataFrame en fichier CSV
df.to_csv('best_cities.csv', index=False)

# analyse : overview des météos

In [41]:
bc = pd.read_csv("best_cities.csv")
bc.head()

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
0,Mont Saint Michel,Mont Saint-Michel,FR,Normandy,48.635954,-1.511460,2025-03-28 09:56:08,9.73,7.64,0.54,93.0,4.07,overcast clouds
1,St Malo,Saint-Malo,FR,Brittany,48.649518,-2.026041,2025-03-28 09:56:26,10.26,9.69,0.40,90.0,3.60,overcast clouds
2,Bayeux,Bayeux,FR,Normandy,49.276462,-0.702474,2025-03-28 09:56:26,9.25,6.52,0.64,93.0,5.34,overcast clouds
3,Le Havre,Le Havre,FR,Normandy,49.493898,0.107973,2025-03-28 09:56:26,10.03,9.41,1.57,89.0,4.12,mist
4,Rouen,Rouen,FR,Normandy,49.440459,1.093966,2025-03-28 09:56:26,10.43,9.67,3.20,82.0,1.79,overcast clouds


In [36]:
bc.shape

(35, 13)

In [37]:
df_sorted = bc.sort_values(by='temp', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='temp', y='cities', orientation='h',
             title='Températures par Ville',
             labels={'temp': 'Température (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [43]:
df_sorted = bc.sort_values(by='feels_like', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='feels_like', y='cities', orientation='h',
             title='Températures ressenties par Ville',
             labels={'feels_like': 'Température ressentie (°C)', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [76]:
df_sorted = bc.sort_values(by='uvi', ascending=True)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='uvi', y='cities', orientation='h',
             title='indices UV par Ville',
             labels={'uvi': 'Indice UV', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [38]:
df_sorted = bc.sort_values(by='humidity', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='humidity', y='cities', orientation='h',
             title='Humidité par Ville',
             labels={'humidity': 'Humidité', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [40]:
df_sorted = bc.sort_values(by='wind_speed', ascending=False)

# Créer un diagramme en bâtons horizontal
fig = px.bar(df_sorted, x='wind_speed', y='cities', orientation='h',
             title='Vitesse du vent par Ville',
             labels={'wind_speed': 'wind_speed', 'cities': 'Ville'})

# Ajuster la hauteur du graphique
fig.update_layout(height=800)

# Afficher le diagramme
fig.show()

In [74]:
# Dictionnaire pour mapper les descriptions aux icônes
icon_mapping = {
    'clear sky': "https://images.icon-icons.com/2791/PNG/512/sunny_day_weather_sun_icon_177555.png",  # Remplacez par l'URL réelle
    'scattered clouds': 'https://images.icon-icons.com/2791/PNG/512/partly_cloudy_day_sun_clouds_weather_icon_177560.png',
    'broken clouds': 'https://images.icon-icons.com/2791/PNG/512/cloudy_cloud_weather_icon_177580.png',
    'overcast clouds': 'https://images.icon-icons.com/2791/PNG/512/overcast_cloud_weather_icon_177561.png',
    'mist': 'https://images.icon-icons.com/2791/PNG/512/fog_weather_icon_177578.png'
}

# Créer une carte centrée sur la France
map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter les villes à la carte avec des icônes personnalisées
for i, row in bc.iterrows():
    icon_url = icon_mapping.get(row['description'], 'https://images.icon-icons.com/1744/PNG/512/3643775-caution-exclamation-mark-sign-triangle_113444.png')  #si non trouvé dans le dico, ce logo apparait
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['cities'],
        icon=folium.CustomIcon(icon_url, icon_size=(40, 40))  # Ajustez la taille de l'icône si nécessaire
    ).add_to(map)

# Sauvegarder la carte dans un fichier HTML
map.save('map_meteo_best_city.html')

# Afficher la carte
map


Nous sommes en mars. Nous cherchons une distination qui nous changerait de la grisaille parisienne. Pour définir les villes avec la meilleure météo, nous pouvons définir les conditions suivantes :
- bc["description"]=="clear sky"
- bc["feels like"]
- bc["uvi]
- bc["humidity"]
- bc["wind speed"]

In [96]:
bc[bc["description"]=="clear sky"]

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
8,Strasbourg,Strasbourg,FR,Grand Est,48.584614,7.750713,2025-03-28 09:56:26,12.49,11.41,3.66,62.0,1.54,clear sky
9,Chateau du Haut Koenigsbourg,Château du Haut-Kœnigsbourg,FR,Grand Est,48.249523,7.345492,2025-03-28 09:56:27,9.44,8.49,3.77,63.0,2.06,clear sky
10,Colmar,Colmar,FR,Grand Est,48.077752,7.357964,2025-03-28 09:56:27,11.89,10.81,3.77,64.0,2.06,clear sky
11,Eguisheim,Eguisheim,FR,Grand Est,48.044797,7.307962,2025-03-28 09:56:27,11.53,10.41,3.77,64.0,2.06,clear sky
12,Besancon,Besançon,FR,Bourgogne-Franche-Comté,47.238022,6.024362,2025-03-28 09:56:27,11.74,10.72,3.84,67.0,1.29,clear sky
13,Dijon,Dijon,FR,Bourgogne-Franche-Comté,47.321581,5.041470,2025-03-28 09:56:27,11.62,10.46,3.84,62.0,1.54,clear sky
17,Bormes les Mimosas,Bormes-les-Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-03-28 09:56:27,15.76,14.98,4.14,61.0,2.57,clear sky
21,Avignon,Avignon,FR,Provence-Alpes-Côte d'Azur,43.949249,4.805901,2025-03-28 09:56:28,15.28,14.25,4.18,53.0,9.77,clear sky
23,Nimes,Nimes,FR,Occitania,43.837425,4.360069,2025-03-28 09:56:28,15.03,13.92,4.15,51.0,5.14,clear sky
24,Aigues Mortes,Aigues-Mortes,FR,Occitania,43.565823,4.191284,2025-03-28 09:56:28,14.96,13.97,4.15,56.0,5.14,clear sky


In [99]:
# Filtrer les villes avec la description "clear sky"
filtered_bc = bc[bc['description'] == 'clear sky']

# Sélectionner les 5 meilleures villes pour chaque critère parmi les villes filtrées
top_feels_like = set(filtered_bc.nlargest(5, 'feels_like')['cities'])
top_temp = set(filtered_bc.nlargest(5, 'temp')['cities'])
top_uvi = set(filtered_bc.nlargest(5, 'uvi')['cities'])
top_humidity = set(filtered_bc.nsmallest(5, 'humidity')['cities'])
#top_wind_speed = set(filtered_bc.nsmallest(5, 'wind_speed')['cities'])

# Trouver l'intersection des ensembles
best_weather = top_feels_like & top_temp & top_uvi & top_humidity 
#& top_wind_speed

# Afficher la liste des meilleures villes
print("Villes avec la meilleure météo :")
print(best_weather)

Villes avec la meilleure météo :
{'Avignon', 'Nimes', 'Aigues Mortes', 'Bormes les Mimosas'}


In [114]:
best_weather = list(best_weather) #transformer le set en liste
best_weather_df = bc[bc['cities'].isin(best_weather)] #puis filter le dataframe sur la liste
best_weather_df

,cities,name,country,state,lat,lon,date,temp,feels_like,uvi,humidity,wind_speed,description
17,Bormes les Mimosas,Bormes-les-Mimosas,FR,Provence-Alpes-Côte d'Azur,43.150697,6.341928,2025-03-28 09:56:27,15.76,14.98,4.14,61.0,2.57,clear sky
21,Avignon,Avignon,FR,Provence-Alpes-Côte d'Azur,43.949249,4.805901,2025-03-28 09:56:28,15.28,14.25,4.18,53.0,9.77,clear sky
23,Nimes,Nimes,FR,Occitania,43.837425,4.360069,2025-03-28 09:56:28,15.03,13.92,4.15,51.0,5.14,clear sky
24,Aigues Mortes,Aigues-Mortes,FR,Occitania,43.565823,4.191284,2025-03-28 09:56:28,14.96,13.97,4.15,56.0,5.14,clear sky


# ANNEXES